In [2]:
!pip install spacy pyodbc

In [3]:
stop_words = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any",
    "are", "aren't", "as", "at", "be", "because", "been", "before", "being", "below",
    "between", "both", "but", "by", "can't", "cannot", "could", "couldn't", "did", "didn't",
    "do", "does", "doesn't", "doing", "don't", "down", "during", "each", "few", "for",
    "from", "further", "had", "hadn't", "has", "hasn't", "have", "haven't", "having", "he",
    "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself",
    "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is",
    "isn't", "it", "it's", "its", "itself", "let's", "me", "more", "most", "mustn't", "my",
    "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought",
    "our", "ours", "ourselves", "out", "over", "own", "same", "shan't", "she", "she'd",
    "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than", "that", "that's",
    "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they",
    "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under",
    "until", "up", "very", "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were",
    "weren't", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who",
    "who's", "whom", "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd",
    "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"
]


In [4]:
sentence = u'Show all students with marks greater than 30'

In [5]:
# basic string replacements for count and sum
sentence = sentence.replace("total number", "count")
sentence = sentence.replace("total", "sum")
sentence

'Show all students with marks greater than 30'

In [6]:
# remove the stop words
new_sentence = ""
for word in sentence.split():
    if word not in stop_words:
        new_sentence += word + " "
sentence = new_sentence.lstrip()
sentence

'Show students marks greater 30 '

In [ ]:
  #  for loaded_entity in db_model.loaded_entities:
        # for loaded_entity_value in loaded_entity[1]:
        #     if loaded_entity_value.lower() in sentence:
        #         sentence = replace_entities(sentence, loaded_entity_value, loaded_entity_value)



In [7]:
import copy


class Matcher(object):
    def __init__(self):
        self.matcher = []
        return super().__init__()

    def add(self, key, value):
        self.matcher.append((key, value))

    def find(self, phrase):
        matches = []
        for match in self.matcher:
            if " " + str(match[1]) + " " in phrase \
                    or phrase.startswith(str(match[1]) + " ") \
                    or phrase.endswith(" " + str(match[1])):
                matches.append(copy.copy(match))
        return matches

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [19]:
doc = nlp(sentence)

In [20]:
identified_spans = []
identified_entities = []

for chunk in doc.noun_chunks:
    identified_spans.append(chunk.text)
    # print(chunk.text, " -- ", chunk.root.text, " -- ", chunk.root.dep_, " -- ", chunk.root.head.text)
for ent in doc.ents:
    identified_entities.append(ent.text)

In [21]:
identified_entities,identified_spans

(['30'], ['Show students'])

In [24]:
lemma_exceptions = ["greater", "less", "than", "more"]
lemmatizedSentence = ''
for token in doc:
    lemmatizedSentence = lemmatizedSentence + (token.text if token.text in lemma_exceptions else token.lemma_) + " "
lemmatizedSentence = lemmatizedSentence.lstrip()

In [25]:
matches = custom_matcher.find(lemmatizedSentence)
matched_entities = []
matched_columns = []
for match in matches:
    if match[0].endswith("TABLE"):
        matched_entities.append(Entities(match[0].replace("_TABLE","")))
        lemmatizedSentence = replace_string(lemmatizedSentence, str(match[1]), match[0].replace("_TABLE",""))
    if match[0].endswith("COLUMN"):
        columnType = [c.type_ for c in db_model.columns if c.name == match[0].replace("_COLUMN","").lower()]
        if len(columnType) > 0:
            columnType = columnType[0]
        matched_columns.append(Columns(match[0].replace("_COLUMN",""), columnType))
        lemmatizedSentence = replace_string(lemmatizedSentence, str(match[1]), match[0].replace("_COLUMN",""))

NameError: name 'custom_matcher' is not defined

In [26]:
class Entities(object):
    def __init__(self, name=None, defaultColumn=None, primaryKey=None, isAverage=None, isMax=None, isMin=None, isCount=None, columns=None, condition=None, value_=None, isSum=None):
        self.name = name
        self.isAverage = isAverage
        self.isMax = isMax
        self.isMin = isMin
        self.isCount = isCount
        self.columns = []
        self.value_ = value_
        self.condition = condition
        self.defaultColumn = defaultColumn
        self.primaryKey = primaryKey
        self.isSum = isSum

In [ ]:
config_data = {
    "sql": {
        "connection_string":""
    },
    "phrase_splitter": " in |in | in| and |and | and| with |with | with| of |of | of",
    "default_columns": {
        "entities": {
            "student":"name",
            "subject":"name",
            "student_mark":"id"
        }
    },
    "entities_to_load": [
        {
            "entity": "subject",
            "column": "name"
        },
        {
            "entity": "student",
            "column": "name"
        }
    ],
    "synonyms": {
        "column": [
            {
                "original": "class",
                "synonyms": [ "standard" ]
            }
        ],
        "table": [
            {
                "original": "student",
                "synonyms": [ "children", "child" ]
            }
        ]
    }
}

In [ ]:
import json
import os


class Singleton(type):
    """
    Define an Instance operation that lets clients access its unique
    instance.
    """

    def __init__(cls, name, bases, attrs, **kwargs):
        super().__init__(name, bases, attrs)
        cls._instance = None

    def __call__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class Configuration(metaclass=Singleton):
    def __init__(self):
        self.data = config_data

    # sql starts
    def get_sql_connection_string(self):
        return self.data["sql"]["connection_string"]

    def get_tables_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '.', 'tables.sql'))) as query:
            return query.read()

    def get_columns_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '.', 'columns.sql'))) as query:
            return query.read()

    def get_FK_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '.', 'foreign_keys.sql'))) as query:
            return query.read()

    def get_PK_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '.', 'primary_keys.sql'))) as query:
            return query.read()

    def get_synonyms(self):
        return self.data["synonyms"]

    def get_phrase_splitter(self):
        return self.data["phrase_splitter"]

    def get_entitites_to_load(self):
        return self.data["entities_to_load"]

    # sql ends

    def get_default_column(self, table_name):
        return self.data["default_columns"]["entities"][table_name]

In [ ]:
class Columns(object):
    def __init__(self, name=None, type_=None, isAverage=None, isMax=None, isMin=None, isCount=None, value_=None, condition=None, isSum=None):
        self.name = name
        self.type_ = type_
        self.isAverage = isAverage
        self.isMax = isMax
        self.isMin = isMin
        self.isCount = isCount
        self.value_ = value_
        self.condition = condition
        self.isSum = isSum

In [ ]:
class Relationship(object):
    def __init__(self, entity1, entity2, column1, column2):
        self.entity1 = entity1
        self.entity2 = entity2
        self.column1 = column1
        self.column2 = column2

In [ ]:
class Synonyms(object):
    def __init__(self, column, synonym):
        self.column = column
        self.synonym = synonym

In [ ]:
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
import pyodbc

In [ ]:
from spacy import displacy

In [ ]:
#load the configuration
config = Configuration()

In [ ]:
config.get_sql_connection_string()

''

In [ ]:

# load the DB Model
db_model = DBModel()
# remove unneccesary words
stan_stop_words = [line.rstrip('\n') for line in open("stopwords.txt")]
# load spacy's english model
nlp = spacy.load('en_core_web_sm')
# exceptions
exceptions = ["between", "more", "than"]
lemma_exceptions = ["greater", "less", "than", "more"]

In [ ]:
import json
import os


class Singleton(type):
    """
    Define an Instance operation that lets clients access its unique
    instance.
    """

    def __init__(cls, name, bases, attrs, **kwargs):
        super().__init__(name, bases, attrs)
        cls._instance = None

    def __call__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class Configuration(metaclass=Singleton):
    def __init__(self):
        self.data = config_data

    # sql starts
    def get_sql_connection_string(self):
        return self.data["sql"]["connection_string"]

    def get_tables_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '..', 'models', 'sql_scripts', 'tables.sql'))) as query:
            return query.read()

    def get_columns_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '..', 'models', 'sql_scripts', 'columns.sql'))) as query:
            return query.read()

    def get_FK_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '..', 'models', 'sql_scripts', 'foreign_keys.sql'))) as query:
            return query.read()

    def get_PK_sql_query(self):
        with open(os.path.abspath(
                os.path.join(os.path.dirname(__file__), '..', 'models', 'sql_scripts', 'primary_keys.sql'))) as query:
            return query.read()

    def get_synonyms(self):
        return self.data["synonyms"]

    def get_phrase_splitter(self):
        return self.data["phrase_splitter"]

    def get_entitites_to_load(self):
        return self.data["entities_to_load"]

    # sql ends

    def get_default_column(self, table_name):
        return self.data["default_columns"]["entities"][table_name]

In [ ]:
def process_sentence(sentence):

    # basic string replacements for count and sum
    sentence = sentence.replace("total number", "count")
    sentence = sentence.replace("total", "sum")

    # remove the stop words
    new_sentence = ""
    for word in sentence.split():
        if word not in stop_words:
            new_sentence += word + " "
    sentence = new_sentence.lstrip()

    for loaded_entity in db_model.loaded_entities:
        for loaded_entity_value in loaded_entity[1]:
            if loaded_entity_value.lower() in sentence:
                sentence = replace_entities(sentence, loaded_entity_value, loaded_entity_value)

    # run nlp on sentence
    doc = nlp(sentence)

    identified_spans = []
    identified_entities = []

    for chunk in doc.noun_chunks:
        identified_spans.append(chunk.text)
        # print(chunk.text, " -- ", chunk.root.text, " -- ", chunk.root.dep_, " -- ", chunk.root.head.text)
    for ent in doc.ents:
        identified_entities.append(ent.text)
        # print(ent.text, ent.start_char, ent.end_char, ent.label_)

    # build the lemma sentence
    lemmatizedSentence = ''
    for token in doc:
        lemmatizedSentence = lemmatizedSentence + (token.text if token.text in lemma_exceptions else token.lemma_) + " "
    lemmatizedSentence = lemmatizedSentence.lstrip()

    # stop word removal
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

    # for chunk in docLemmatized.noun_chunks:
    #     print(chunk.text, chunk.root.text, chunk.root.dep_,
    #             chunk.root.head.text)

    # get all tables and columns in the question
    # matches = matcher(docLemmatized)
    matches = custom_matcher.find(lemmatizedSentence)
    matched_entities = []
    matched_columns = []
    for match in matches:
        if match[0].endswith("TABLE"):
            matched_entities.append(Entities(match[0].replace("_TABLE","")))
            lemmatizedSentence = replace_string(lemmatizedSentence, str(match[1]), match[0].replace("_TABLE",""))
        if match[0].endswith("COLUMN"):
            columnType = [c.type_ for c in db_model.columns if c.name == match[0].replace("_COLUMN","").lower()]
            if len(columnType) > 0:
                columnType = columnType[0]
            matched_columns.append(Columns(match[0].replace("_COLUMN",""), columnType))
            lemmatizedSentence = replace_string(lemmatizedSentence, str(match[1]), match[0].replace("_COLUMN",""))

    docLemmatized = nlp(lemmatizedSentence)

    # print("####################")
    # print(lemmatizedSentence)
    # print([(m[0], m[1]) for m in custom_matcher.matcher])
    # print([(m[0], m[1]) for m in matches])
    # print([m.name for m in matched_entities])
    # print([m.name for m in matched_columns])
    # print("####################")

    # get values for the captured columns in the above use case
    for token in docLemmatized:

        # check of token matches any of the matched entities
        if token.text.upper() in [m.name for m in matched_entities]:
            matched_entity = next(me for me in matched_entities if me.name == token.text.upper())

            contextual_span = get_neighbour_tokens(token)
            span_ranges = re.split(config.get_phrase_splitter(), contextual_span)
            for span in span_ranges:
                if matched_entity.name.lower() in span:
                    matched_entity.condition = "="
                    if "average" in span:
                        matched_entity.isAverage = True
                    if "avg" in span:
                        matched_entity.isAverage = True
                    if "maximum" in span:
                        matched_entity.isMax = True
                    if "max" in span:
                        matched_entity.isMax = True
                    if "minimum" in span:
                        matched_entity.isMin = True
                    if "min" in span:
                        matched_entity.isMin = True
                    if "count" in span:
                        matched_entity.isCount = True
                    if "sum" in span:
                        matched_entity.isSum = True
                    if "total" in span:
                        matched_entity.isSum = True

                    trimmed_span = span \
                        .replace("average", "") \
                        .replace("maximum", "") \
                        .replace("minimum", "") \
                        .replace("greater than", "") \
                        .replace("less than", "") \
                        .replace("more than", "") \
                        .replace("min", "") \
                        .replace("max", "") \
                        .replace("count", "") \
                        .replace("sum", "")
                    trimmed_span = ' '.join(trimmed_span.split())
                    doc_span = nlp(trimmed_span)

                    for span_token in doc_span:

                        if span_token.text.lower() == matched_entity.name.lower():

                            if get_token_child_len(span_token) > 0:
                                span_token_child = next(itertools.islice(span_token.children, 1))
                                ent = next(en for en in db_model.entities if en.name.lower() == matched_entity.name.lower())
                                default_column = next(col for col in ent.columns if col.name.lower() == ent.defaultColumn.lower())
                                value = get_value(span_token_child.text, default_column.type_)

                                identified_entity_exists = False
                                for identified_entity in identified_entities:
                                    if identified_entity in trimmed_span and str(value) in identified_entity:
                                        identified_entity_exists = True
                                        value = identified_entity
                                matched_entity.value_ = value



            matched_entities = [me for me in matched_entities if me.name != token.text.upper()]
            matched_entities.append(matched_entity)


        # check of token matches any of the matched column
        if token.text.upper() in [m.name for m in matched_columns]:
            matched_column = next(mc for mc in matched_columns if mc.name == token.text.upper())

            contextual_span = get_neighbour_tokens(token)
            span_ranges = re.split(config.get_phrase_splitter(), contextual_span)
            for span in span_ranges:
                # print("column : ", span)
                if matched_column.name.lower() in span:
                    matched_column.condition = "="
                    if "average" in span:
                        matched_column.isAverage = True
                    if "avg" in span:
                        matched_column.isAverage = True
                    if "maximum" in span:
                        matched_column.isMax = True
                    if "max" in span:
                        matched_column.isMax = True
                    if "minimum" in span:
                        matched_column.isMin = True
                    if "min" in span:
                        matched_column.isMin = True
                    if "greater than" in span:
                        matched_column.condition = ">"
                    if "more than" in span:
                        matched_column.condition = ">"
                    if "less than" in span:
                        matched_column.condition = "<"
                    if "count" in span:
                        matched_column.isCount = True
                    if "sum" in span:
                        matched_column.isSum = True
                    if "total" in span:
                        matched_column.isSum = True

                    trimmed_span = span \
                        .replace("average", "") \
                        .replace("maximum", "") \
                        .replace("minimum", "") \
                        .replace("greater than", "") \
                        .replace("less than", "") \
                        .replace("more than", "") \
                        .replace("min", "") \
                        .replace("max", "") \
                        .replace("count", "") \
                        .replace("sum", "")
                    trimmed_span = ' '.join(trimmed_span.split())

                    doc_span = nlp(trimmed_span)

                    for span_token in doc_span:
                        if span_token.text.lower() == matched_column.name.lower():
                            if get_token_child_len(span_token) > 0:
                                span_token_child = next(itertools.islice(span_token.children, 1))
                                value = get_value(span_token_child.text, matched_column.type_)

                                identified_entity_exists = False
                                for identified_entity in identified_entities:
                                    if identified_entity in trimmed_span and str(value) in identified_entity and get_value(identified_entity, matched_column.type_) != "NoValue":
                                        identified_entity_exists = True
                                        value = identified_entity
                                matched_column.value_ = value


            matched_columns = [mc for mc in matched_columns if mc.name != token.text.upper()]
            matched_columns.append(matched_column)

    for loaded_entity in db_model.loaded_entities:
        entity_name = loaded_entity[0]
        for loaded_entity_value in loaded_entity[1]:
            if loaded_entity_value.lower() in lemmatizedSentence.lower():
                if entity_name.lower() in [me.name.lower() for me in matched_entities]:
                    # already exists
                    # no In operator support as of now
                    print("entity already processed")
                else:
                    en_def_col = next(col for en in db_model.entities if en.name.lower() == entity_name.lower() for col in en.columns if col.name.lower() == en.defaultColumn.lower())
                    if get_value(loaded_entity_value, en_def_col.type_) != "NoValue":
                        ent = Entities(entity_name.upper())
                        ent.condition = "="
                        ent.value_ = get_value(loaded_entity_value, en_def_col.type_)
                        matched_entities.append(ent)

    # final representation of columns (matched_columns) and entities (matched_entities), including max, min, average, conditions
    # now next is to build the SQL query generator
    # matched entities
    # print("####################")
    # print("\n".join([(mc.name + " -- " + str(mc.value_) + " -- " + " condition : " + str(mc.condition) + " -- " + " isMax : " + str(mc.isMax) + " -- " + " isMin : " + str(mc.isMin) + " -- " + " isAverage : " + str(mc.isAverage) + " -- " + " isSum : " + str(mc.isSum) + " -- " + " isCount : " + str(mc.isCount)) for mc in matched_entities]))
    # print("####################")
    # matched columns
    # print("\n".join([(mc.name + " -- " + str(mc.value_) + " -- " + " condition : " + str(mc.condition) + " -- " + " isMax : " + str(mc.isMax) + " -- " + " isMin : " + str(mc.isMin) + " -- " + " isAverage : " + str(mc.isAverage)) for mc in matched_columns]))
    # print("####################")

    sql_generator = SQLGenerator(matched_entities, matched_columns, db_model)
    print("=================================================================================")
    result = sql_generator.get_sql()
    # print(sql_generator.query)
    # print("=================================================================================")
    # print(result)
    # print("=================================================================================")
    response = {}
    response['sql'] = sql_generator.query
    response['result'] = result[0]
    response['columns'] = result[1]
    return response


In [ ]:
# remove the stop words
new_sentence = ""
for word in sentence.split():
    if word not in stop_words:
        new_sentence += word + " "
sentence = new_sentence.lstrip()
sentence

'Show students marks greater 30 '

In [ ]:
loaded_entities = [('subject', ['Lokesh Lal', 'Lokesh', 'Lal', 'Munish Sharma', 'Munish', 'Sharma', 'Sumit Chharia', 'Sumit', 'Chharia', 'Manoj Gupta', 'Manoj', 'Gupta', 'Vishal Gupta', 'Vishal', 'Gupta', 'Ambika Shukla', 'Ambika', 'Shukla', 'Manoj Garg', 'Manoj', 'Garg', 'Gagandeep sampat', 'Gagandeep', 'sampat'])]


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
doc = nlp(sentence)
identified_spans = []
identified_entities = []

for chunk in doc.noun_chunks:
    identified_spans.append(chunk.text)
    print(chunk.text, " -- ", chunk.root.text, " -- ", chunk.root.dep_, " -- ", chunk.root.head.text)
for ent in doc.ents:
    identified_entities.append(ent.text)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Show students  --  students  --  nsubj  --  marks
30 28 30 CARDINAL


In [ ]:
exceptions = ["between", "more", "than"]
lemma_exceptions = ["greater", "less", "than", "more"]

lemmatizedSentence = ''
for token in doc:
    lemmatizedSentence = lemmatizedSentence + (token.text if token.text in lemma_exceptions else token.lemma_) + " "
lemmatizedSentence = lemmatizedSentence.lstrip()
lemmatizedSentence

'show student mark greater 30 '

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS